In [1]:
class WhileLoop:
    def __init__(self, condition, content, code_before_while, all_code_after_while):
        self.condition = condition
        self.content = content
        self.code_before_while = code_before_while
        self.all_code_after_while = all_code_after_while

In [2]:
import re

def extract_single_outer_while(source):
    indent_regex = re.compile(r'^\s+', flags=re.MULTILINE)
    try: 
        code_before_while = re.sub(indent_regex, '', source[:source.index('while')].strip())
    except ValueError:
        return WhileLoop(None, None, source, None)

    condition_start = re.search('while\s*\(', source).end()
    condition_end = source.index(')', condition_start)
    condition = source[condition_start:condition_end].strip()
    
    # search for the end } by matching the number of { and } in the content
    content_start = source.index('{', condition_end) + 1
    brace_count = 1
    for i, c in enumerate(source[content_start:]):
        if c == '{':
            brace_count += 1
        elif c == '}':
            brace_count -= 1
            if brace_count == 0:
                content_end = content_start + i
                break

    content = re.sub(indent_regex, '', source[content_start:content_end].strip())
    all_code_after_while = re.sub(indent_regex, '', source[content_end+1:].strip())
    return WhileLoop(condition, content, code_before_while, all_code_after_while)

In [3]:
# test for extract_single_outer_while(source)
with open('source.c', 'r') as f:
    source = f.read()
loop = extract_single_outer_while(source)
print("================================")
print(loop.condition)
print("================================")
print(loop.content)
print("================================")
print(loop.code_before_while)
print("================================")
print(loop.all_code_after_while)

y <= y2
int x = x1;
while (x <= xR)
{
setPixel(x, y, 1);
x++;
}
while (errR < 0)
{
xR++;
errR = errR + 2 * (y2 - y1);
}
errR = errR - 2 * (x2 - x1);
y++;
int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
int counter = 0;
while (counter < 10)
{
counter++;
}
counter *= 2;


In [4]:
def extract_multiple_outer_while(source):
    # (node, depth)
    all_loops = []
    all_loops.append((extract_single_outer_while(source), 0))
    while all_loops[-1][0].all_code_after_while and all_loops[-1][0].all_code_after_while.strip():
        all_loops.append((extract_single_outer_while(all_loops[-1][0].all_code_after_while), all_loops[-1][1]+1))
    return all_loops

In [5]:
# test for extract_multiple_outer_while(source)
with open('source.c', 'r') as f:
    source = f.read()
loops = extract_multiple_outer_while(source)
for loop in loops:
    print(loop[0].code_before_while)
    if (loop[0].condition): 
        print("while (", loop[0].condition, ") {")
        print(loop[0].content)
        print("}")


int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
while ( y <= y2 ) {
int x = x1;
while (x <= xR)
{
setPixel(x, y, 1);
x++;
}
while (errR < 0)
{
xR++;
errR = errR + 2 * (y2 - y1);
}
errR = errR - 2 * (x2 - x1);
y++;
}
int counter = 0;
while ( counter < 10 ) {
counter++;
}
counter *= 2;


In [13]:
class Node:
    def __init__(self, data, children, condition=False):
        self.data = data
        self.children = children
        self.condition = condition

def build_tree(content):
    if (extract_single_outer_while(content).condition == None):
        return Node(content, [])
    all_loops = []
    all_loops.append(extract_single_outer_while(content))
    while all_loops[-1].all_code_after_while and all_loops[-1].all_code_after_while.strip():
        all_loops.append(extract_single_outer_while(all_loops[-1].all_code_after_while))
    children = []
    for loop in all_loops:
        if loop.content: 
            children.append(Node(loop.code_before_while, []))
            children.append(Node(loop.condition, [build_tree(loop.content)], True))
        else: 
            children.append(Node(loop.code_before_while, []))
    return Node('', children)

# test for build_tree(loop)
with open('source.c', 'r') as f:
    source = f.read()
tree = build_tree(source)

def print_tree(node, indent=-1):
    # print('  '*indent + node.data)
    # print with indent in every line in node.data
    if node.condition:
        print('  '*indent + 'while (' + node.data + ') {')
    else: 
        for line in node.data.splitlines():
            print('  '*indent + line)
    for child in node.children:
        print_tree(child, indent+1) 
    if node.condition:
        print('  '*indent + '}')

# test for print_tree(tree)
print_tree(tree)

int errR = (y2 - y1) - 2 * (x2 - x1);
int xR = x1;
int y = y1;
while (y <= y2) {
    int x = x1;
    while (x <= xR) {
      setPixel(x, y, 1);
      x++;
    }
    while (errR < 0) {
      xR++;
      errR = errR + 2 * (y2 - y1);
    }
    errR = errR - 2 * (x2 - x1);
    y++;
}
int counter = 0;
while (counter < 10) {
  counter++;
}
counter *= 2;
